In [ ]:
# This bertopic reduces and incresaes the number of topics for models that are above a ceratin threshold or under 

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from umap import UMAP
from hdbscan import HDBSCAN
import pandas as pd

# Load the dataset
data = pd.read_csv('mozilla_core_updated.csv')
fixed_bugs = data[data['Resolution'] == 'FIXED']
fixed_bugs = fixed_bugs[fixed_bugs['Assignee Real Name'].notna()]
devNum = fixed_bugs['Assignee Real Name'].value_counts()
devNum = devNum[(devNum > 600) & (~devNum.index.str.startswith('Platform')) & (~devNum.index.str.startswith('platform'))]
print(devNum)
print(len(devNum))

def devDocList(name, data):
    docs = data[data['Assignee Real Name'] == name]
    docs = docs[['Title','Description','Product', 'Component', 'Priority', 'Severity']]
    docs['Title'] = docs['Title'].fillna('')
    docs['Description'] = docs['Description'].fillna('')
    docs['Product'] = docs['Product'].fillna('')
    docs['Component'] = docs['Component'].fillna('')
    docs['Priority'] = docs['Priority'].fillna('') # Just added to test
    docs['Severity'] = docs['Severity'].fillna('')  # Just added to test
    docs['Text'] = docs['Product'] + ' ' + docs['Component'] + ' ' + docs['Title']  + ' ' + docs['Description'] + ' ' + docs['Priority'] + ' ' + docs['Severity']
    return docs

# 1. Embedding
sentence_model = SentenceTransformer('all-mpnet-base-v2')

# 2. Dimensionality Reduction
umap_model = UMAP(n_components=1,
                  n_neighbors=15,
                  min_dist=0.0,
                  metric='correlation')

# 3. Clustering 
hdbscan_model = HDBSCAN(min_cluster_size=20,
                        min_samples=1, 
                        metric='euclidean', 
                        cluster_selection_method='leaf',
                        prediction_data=True,
                        allow_single_cluster=True)

# 4. Token
vec = CountVectorizer(stop_words='english',
                      ngram_range=(1,2))

# 5. Weighting schemes
ctif = ClassTfidfTransformer(reduce_frequent_words=True)

# 6. Fine Tune
representation = KeyBERTInspired()

test_documents = []

count = 0

for dev in devNum.index:

    print(f"Training model for {dev}...")
   
    docs_df = devDocList(dev, data)
   
    docTrain_df, docTest_df = train_test_split(docs_df, test_size=0.2, shuffle=False)
   
    docTest_df['Developer'] = dev
    test_documents.append(docTest_df)
   
    docTrain = docTrain_df['Text'].tolist()
   
    # Create and fit BERTopic model
    topic_model = BERTopic(embedding_model=sentence_model,          #1
                           umap_model=umap_model,                   #2
                           hdbscan_model=hdbscan_model,             #3
                           vectorizer_model=vec,                    #4
                           ctfidf_model=ctif,                       #5
                           representation_model=representation)     #6

    topics, probs = topic_model.fit_transform(docTrain)
   
    # Check the number of topics
    topic_info = topic_model.get_topic_info()
    num_topics = len(topic_info) - 1 # Exclude the outlier topic
   
    if num_topics > 10:
        topic_model.reduce_topics(docs=docTrain, nr_topics=8)
    elif num_topics < 1:
        # Adjust HDBSCAN parameters to reduce outliers
        hdbscan_model = HDBSCAN(min_cluster_size=5,
                                min_samples=1, 
                                metric='euclidean', 
                                cluster_selection_method='leaf',
                                prediction_data=True,
                                allow_single_cluster=False)
        
        topic_model = BERTopic(embedding_model=sentence_model,          #1
                               umap_model=umap_model,                   #2
                               hdbscan_model=hdbscan_model,             #3
                               vectorizer_model=vec,                    #4
                               ctfidf_model=ctif,                       #5
                               representation_model=representation)     #6
        
        topics, probs = topic_model.fit_transform(docTrain)
   
    # Print topic information
    print(topic_model.get_topic_info())
    print(topic_model.get_document_info(docTrain))
    print(probs.mean())
   
    #Save the topic model
    topic_model.save(f'{count}_{dev}')
   
    count += 1

print("All models trained and saved.")

# Concatenate all test documents into a single DataFrame
test_documents_df = pd.concat(test_documents, ignore_index=True)

# Save the test documents to a CSV file
test_documents_df.to_csv('test_documents_platform.csv', index=False)


Assignee Real Name
Unassigned           7979
bzbarsky             3522
roc                  1776
dbaron               1696
brendan              1222
mrbkap               1217
smaug                1149
ehsan.akhgari        1115
dougt                1107
timeless             1050
jst                  1044
Ms2ger                983
darin.moz             976
surkov                972
dholbert              911
MatsPalmgren_bugz     867
glandium              850
aaronlev              798
jaas                  782
peterv                775
rods                  768
pavlov                718
benjamin              706
n.nethercote          705
hyatt                 703
karnaze               691
igor                  685
masayuki              651
buster                642
Waldo                 629
mounir                619
jandem                619
Name: count, dtype: int64
32
Training model for Unassigned...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

   Topic  Count                                               Name  \
0     -1   4525  -1_thefirefoxproduct info_thefirefoxproduct_mo...   
1      0  32647  0_thefirefoxproduct info_thefirefoxproduct_fir...   
2      1   2752  1_bug productwatch_core graveyardcore_bugs pro...   
3      2    536  2_firefox build_systemfirefox build_build syst...   
4      3    147  3_developer infrastructuredeveloper_bugs produ...   
5      4    100  4_checker spellchecker_spellchecker_spell chec...   
6      5     26  5_invalid bugsinvalid_bugsinvalid bugsfor_bugs...   
7      6     20  6_cycle collection_node cycle_cycle collection...   

                                      Representation  \
0  [thefirefoxproduct info, thefirefoxproduct, mo...   
1  [thefirefoxproduct info, thefirefoxproduct, fi...   
2  [bug productwatch, core graveyardcore, bugs pr...   
3  [firefox build, systemfirefox build, build sys...   
4  [developer infrastructuredeveloper, bugs produ...   
5  [checker spellchecker, spellch

2025-04-16 18:49:39,480 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


   Topic  Count                                               Name  \
0     -1    233  -1_thefirefoxproduct_thefirefoxproduct info_mo...   
1      0   1502  0_thefirefoxproduct info_thefirefoxproduct_cor...   
2      1    891  1_thefirefoxproduct_thefirefoxproduct info_fir...   
3      2    133  2_bug_0x0 upto_bin_file_version_linker_version...   
4      3     59         3_html charset_xmlhttp_charset utf_mozilla   
5      4     35    4_mozilla svg_svg element_svg load_svg document   
6      5     25  5_xpcom_mem_leak_log_leakage_thefirefoxproduct...   
7      6     21                 6_mozilla_fx mozilla1_mozilla1_bug   

                                      Representation  \
0  [thefirefoxproduct, thefirefoxproduct info, mo...   
1  [thefirefoxproduct info, thefirefoxproduct, co...   
2  [thefirefoxproduct, thefirefoxproduct info, fi...   
3  [bug, 0x0 upto_bin_file_version, linker_versio...   
4  [html charset, xmlhttp, charset utf, mozilla, ...   
5  [mozilla svg, svg element, svg

...

2025-04-16 19:03:05,975 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


   Topic  Count                                               Name  \
0     -1     46  -1_thefirefoxproduct_thefirefoxproduct info_fi...   
1      0    239  0_core html_corecoreshared components_dom core...   
2      1     64  1_core html_thefirefoxproduct info_thefirefoxp...   
3      2     57  2_thefirefoxproduct_firefox user_thefirefoxpro...   
4      3     35  3_systemfirefox build_build systemfirefox_fire...   
5      4     28  4_test browser_bug561636_browser_bug561636_bro...   
6      5     24  5_progress element_progress bars_progress bar_...   
7      6     23  6_core html_html nshtmlinputelement_dom core_n...   

                                      Representation  \
0  [thefirefoxproduct, thefirefoxproduct info, fi...   
1  [core html, corecoreshared components, dom cor...   
2  [core html, thefirefoxproduct info, thefirefox...   
3  [thefirefoxproduct, firefox user, thefirefoxpr...   
4  [systemfirefox build, build systemfirefox, fir...   
5  [test browser_bug561636, brows

2025-04-16 19:03:16,627 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


   Topic  Count                                               Name  \
0     -1     88  -1_thefirefoxproduct info_thefirefoxproduct_cp...   
1      0    159  0_thefirefoxproduct info_thefirefoxproduct_moz...   
2      1    107  1_engine ionmonkey_js ion_thefirefoxproduct_th...   
3      2     53  2_thefirefoxproduct info_bug productwatch_mozi...   
4      3     35  3_javascript engine_thefirefoxproduct_micro be...   
5      4     27  4_engine ionmonkey_javascript engine_ionscript...   
6      5     25  5_thefirefoxproduct_javascript engine_thefiref...   
7      6     20  6_mozilla x11_firefox user_javascript engine_t...   

                                      Representation  \
0  [thefirefoxproduct info, thefirefoxproduct, cp...   
1  [thefirefoxproduct info, thefirefoxproduct, mo...   
2  [engine ionmonkey, js ion, thefirefoxproduct, ...   
3  [thefirefoxproduct info, bug productwatch, moz...   
4  [javascript engine, thefirefoxproduct, micro b...   
5  [engine ionmonkey, javascript 

: 